<a href="https://colab.research.google.com/github/csar95/hugging-face-course/blob/main/workbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import openai
from pytube import YouTube

In [2]:
# Load your API key from an environment variable or secret management service
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# Ask user for the YouTube video URL
yt_url = "https://www.youtube.com/watch?v=VRm2UMsFVz0&list=PLbUIzNzCKD8CUVjPwFAUoNDc0souCNHLs&index=1"

# Create a YouTube object from the URL
yt = YouTube(yt_url)

In [4]:
# Get the audio stream
audio_stream = yt.streams.filter(only_audio=True).first()

In [5]:
# Download the audio stream
output_path = "data"
filename = f"{yt.title}.mp3"
audio_stream.download(output_path=output_path, filename=filename)

'/Users/csr95/Desktop/Mis_Documentos/Code_Python/whispering-key-points/data/ACID Transactions.mp3'

In [6]:
language = 'en'

In [7]:
%%time

with open(f"{output_path}/{filename}", "rb") as audio_file:
    transcript = openai.Audio.transcribe("whisper-1", audio_file, language)

CPU times: user 21.7 ms, sys: 10.6 ms, total: 32.4 ms
Wall time: 16.7 s


In [9]:
transcript.text

"Hello, my name is Niklas and today we're going to talk about transactions in a relational database management system. More specifically, we're going to talk about the ACID acronym, what it stands for and what it actually means. So fundamentally, a transaction is a unit of work executed to retrieve, insert, remove and or update data. And this unit of work typically consists of running one or more statements or queries against some kind of database. In a relational database management system, transactions come with specific properties that serve as guarantees of sorts. So we can rely on the fact that all transactions in a relational database management system will be atomic, they will be consistent, they will be isolated and they will be durable or ACID for short. It's worth mentioning that while all relational database management systems have these guarantees, not all database management systems do so. So some have base properties instead of ACID, but we won't go into this in this vide